### 2020-12-22 Tuesday
### 装饰器的作用是在不改变被装饰函数（以下用f指代）的代码，
### 以及调用f的代码的情况下，给f添加额外的功能。
### 首先定义一个“待装饰”的简单函数。如下：

In [27]:
def myfunc_sum(*nums):
    print("I'm the first function to be decorated.\r")
    val_sum = sum(nums)
    print(f"I sum up all numeric arguments passed and return {val_sum}.")
myfunc_sum(1,2,3)

I'm the first function to be decorated.
I sum up all numeric arguments passed and return 6.


### 现在想要了解上面的这个函数运行时耗时多少；当然可以直接改写这个函数的定义，把计时部分加进去，如下：

In [23]:
import time
def myfunc_sum_timeit(*nums):
    time_start = time.time()
    print("I'm the first function to be decorated.\r")
    val_sum = sum(nums)
    print(f"I sum up all numeric arguments passed and return {val_sum}.")
    time_end = time.time()
    print(f"Time elapsed: {time_end - time_start} seconds.")
#myfunc_sum_timeit(1,2,3,4)

I'm the first function to be decorated.
I sum up all numeric arguments passed and return 10.
Time elapsed: 0.00027179718017578125 seconds.


### 这样确实可以，但需要修改函数定义本身，非常麻烦。
### 能想到的一个方法，就是定义另一个函数来定时，其中调用上述我们感兴趣的函数。如下：

In [28]:
def timeit(func, *nums):
    time_start = time.time()
    func(*nums)
    time_end = time.time()
    print(f"Time elapsed: {time_end - time_start} seconds.")
timeit(myfunc_sum, 1,2,3,4,5)

I'm the first function to be decorated.
I sum up all numeric arguments passed and return 15.
Time elapsed: 0.00044274330139160156 seconds.


### 上述做法达到了”不改动被调用函数本身“的目的，但调用部分的代码变了；
### 从原来的"myfunc_sum(*nums)"变成了"timeit(myfunc_sum, *nums)"。
### 这样，在调用处就要逐一修改，麻烦且易有遗漏。

### 能否既不改动被调用函数本身，又不改动调用代码？
### 以本案为例，很自然的思路是将timeit赋值给myfunc_sum，
### 然后直接调用新的myfunc_sum, 即原来的myfunc_sum与外部计时部分的组合:

In [29]:
myfunc_sum = timeit
myfunc_sum(1,2,3)

TypeError: 'int' object is not callable

### 上述方法报错：
### "TypeError: 'int' object is not callable"
### 也就是说，现在myfunc_sum已经不是初始定义的那个函数myfunc_sum了；
### 它现在与timeit函数完全等价。
### 那么，现在调用myfunc_sum(*sum)时，其实就是在调用timeit(*sum);
### 但timeit的第一个参数应该是一个函数，这里没有；所以抛出上述错误。

### 针对这一点，可以考虑在timeit里再定义一个函数，这个新函数有如下的特点：
#### 1. “封装”了原timeit中计时和调用myfunc_sum函数的两部分；
#### 2. 与myfunc_sum的参数完全一致。
### 然后返回这个新函数。
### 这样，调用timeit(myfunc_sum)并赋值给原myfunc_sum后，myfunc_sum就与这个被返回的函数完全等价了；
### 再调用新的myfunc_sum，就达到了既不更改myfunc_sum本身的代码，也不更改调用代码的目的。如下：


In [32]:
def myfunc_sum_new(*nums):
    '''
    Defining the function to be decorated,
    with arguments of variable numbers.
    '''
    print("I'm the first function to be decorated.\r")
    val_sum = sum(nums)
    print(f"I sum up all numeric arguments passed and return {val_sum}.")
#myfunc_sum_new(1,2,3,4,5)

def timeit_new(func):
    def wrapper(*nums):
        time_start = time.time()
        func(*nums)
        time_end = time.time()
        print(f"Time elapsed: {time_end - time_start} seconds.")
    return wrapper

myfunc_sum_new = timeit_new(myfunc_sum_new)
myfunc_sum_new(1,2,3,4)

I'm the first function to be decorated.
I sum up all numeric arguments passed and return 10.
Time elapsed: 0.0007176399230957031 seconds.


### 由以上代码的实现结果可见，myfunc_sum_new在经过timeit_new“处理”后，
### 达到了在没有改动原函数定义代码和调用代码的情况下，添加了额外功能的目的。

### 在Python中，有一种更“高端”的写法，也就是所谓的语法糖。
### 具体实施方法为：先定义装饰器函数，然后@该函数，在@语句下方定义需要装饰的函数；
### 最后直接调用被装饰的函数即可。如下：

In [34]:
def timeit_dec(func):
    def wrapper(*nums):
        time_start = time.time()
        func(*nums)
        time_end = time.time()
        print(f"Time elapsed: {time_end - time_start} seconds.")
    return wrapper

@timeit_dec
def myfunc_sum_dec(*nums):
    print("I'm the first function to be decorated.\r")
    val_sum = sum(nums)
    print(f"I sum up all numeric arguments passed and return {val_sum}.")

myfunc_sum_dec(1,2,3,4,5,6)

I'm the first function to be decorated.
I sum up all numeric arguments passed and return 21.
Time elapsed: 0.0006966590881347656 seconds.


### 2020-12-23 Wednesday
### 由以上结果可见，用简洁的“语法糖”实现了我们所需要的结果。
### 下面再举一个简单的实例，即“计算结果加密”；显示一下如何正确装饰有返回值的函数。

In [40]:
def encryption_dec(func):
    c = 150
    d = 200
    def wrapper(*nums):
        val_func = func(*nums)
        val_encrypted = val_func*c/d
        '''
        str_returned = "My ENCRYPTED returned value is %.2f."%(val_encrypted)
        return str_returned
        '''
        print(f"My ENCRYPTED returned value is {val_encrypted}.")
    return wrapper

@encryption_dec
def myfunc_sum_return(*nums):
    print("I'm the second function to be decorated.\n"
          "My returned value is to be encrypted.\n")
    val_sum = sum(nums)
    return val_sum

#print(myfunc_sum_return(1,2,3))
myfunc_sum_return(1,2,3)

I'm the second function to be decorated.
My returned value is to be encrypted.

My ENCRYPTED returned value is 4.5.


### 简单计算一下，若输入的不定参数为（1，2，3），则相加和为6，“加密”（乘以150，再除以200）后的结果为4.5. 结果正确。
### 值得注意的是，“Python高级编程——装饰器Decorator超详细讲解(中篇)”一篇中这里讲得有不严谨之处。
### 事实上，待装饰函数有返回值时，wrapper函数完全可以没有返回值；只要正确处理了待装饰函数的返回值即可。如上所示。

### 以上即为用函数装饰器装饰函数的介绍。总结一下，即定义一个装饰器外层，把待装饰的函数设为待传入的参数。
### 内层的wrapper为实际“接收”该函数并进行对其进行“装饰”的“处理器”。需要注意的是，wrapper的参数列表要与待装饰的函数一致，否则后续传参时会发生错误。

### Python中“一切皆对象”；因此，类也可以作为参数传入函数。利用这一性质，可定义用于“装饰”类的函数。
### 下面是关于用函数装饰器装饰类的介绍。

### 在软件设计模式中，有一种类的“单例模式”，即保证该类只有一个实例存在
### （普通类每次实例化，各个实例都是不同的对象，在内存中的地址也不同，可以用id()函数查看；单例模式下，无论多少次实例化，各个实例都是同一个，占据同一个内存地址）。
### 单例模式可以在类的定义中实现；但如果有已经定义了的类，不想/不能改变定义语句而又想额外添加单例模式限制，也可用很简单的函数装饰器加以实现。如下：

In [75]:
def singleton_dec(cls):
    instance = {}
    def wrapper(*args, **kwargs):
        if cls not in instance:
            instance[cls] = cls(*args, **kwargs)
        return instance[cls]
    return wrapper

@singleton_dec
class studentInfo:
    def __init__(self,name,gender,age):
        self.name = name
        self.gender = gender
        self.age = age

    def print_info(self):
        print(f"Name: {self.name}\nGender: {self.gender}\nAge: {self.age}\n")

student_1 = studentInfo("Kevin","male",33)
student_2 = studentInfo("Vincent","male",38)

student_2.print_info()
print(student_1 == student_2)
print(id(student_1)==id(student_2))

Name: Kevin
Gender: male
Age: 33

True
True


### 由以上结果可见，在用“单例化”函数修饰器对“学生信息”类添加单例限制后，该类只能实例化一次；
### 再次实例化并引入新的初始化函数参数，并不会对第一次生成的实例产生影响。两者的内存地址也完全一致。
### 若将语法糖注释掉，再运行一遍上面的cell，“学生信息”就是普通的类了；多次实例化会生成各不相同，内存地址也不同的多个对象。
### 由此也可见装饰器（语法糖）的便利；不需要添加的额外功能时，将@语句注释掉，然后重新运行即可，而不需修改其他任何语句。

### 下面再举一个例子，如何用函数装饰器给类添加新的属性。如下：

In [76]:
def addattr_dec(cls):
    height_cm = 172
    weight_kg = 70
    def wrapper(*args,**kwargs):
        obj_new = cls(*args, **kwargs)
        obj_new.height_cm = height_cm
        obj_new.weight_kg = weight_kg
        return obj_new
    return wrapper

@addattr_dec
class studentInfo_2:
    def __init__(self,name,gender,age):
        self.name = name
        self.gender = gender
        self.age = age

student_3 = studentInfo_2("Maria","female",33)
student_4 = studentInfo_2("Lotus","female",33)
student_4.height_cm = 163
student_4.weight_kg = 65
print(student_3.__dict__)
print("\n")
print(student_4.__dict__)
#type(student_4)

{'name': 'Maria', 'gender': 'female', 'age': 33, 'height_cm': 172, 'weight_kg': 70}


{'name': 'Lotus', 'gender': 'female', 'age': 33, 'height_cm': 163, 'weight_kg': 65}


### 通过函数装饰器，给类studentInfo_2添加了除该类本身定义的名字、性别和年龄三个属性之外的身高、体重两个新属性。
### 函数装饰器的简介先讲到这里。下面来看一下类装饰器。类装饰器同样既可以装饰函数，也可以装饰类。先看第一种情况。
### 用类装饰器装饰函数时，先定义该类并将待装饰函数作为参数传入初始化方法，初始化为该类的属性。
### 然后，定义__call__魔法方法（该方法的作用是使得实例化的类能像函数一样直接调用），在该方法中书写添加功能的语句并调用待修饰函数即可。如下：
### （注意，这里的魔法方法参数要与待装饰函数统一。不再赘述）

In [50]:
class funcDecorator:
    def __init__(self,func):
        self.func = func
    def __call__(self):
        print("Starting...")
        time.sleep(2)
        self.func()
        time.sleep(2)
        print("The End.")

@funcDecorator
def myfunc_class():
    print("I'm the first function to be decorated by a class decorator.")

myfunc_class()

Starting...
I'm the first function to be decorated by a class decorator.
The End.


### 这里，语法糖起到的作用等价于myfunc_class = funcDecorator(myfunc_class);
### 也就是说，最后实际被调用执行的，并不是函数myfunc_class(), 而是实例化的类funcDecorator(myfunc_class).
### 魔法方法__call__则使得该实例对象像函数一样被调用。

### 下面再举一个用类装饰器装饰函数的例子。仍以上述的装饰器对计算结果加密为例。如下：

In [54]:
class funcEncryption:
    def __init__(self,func):
        self.func = func
        self.c = 150
        self.d = 200
    def __call__(self, *args, **kwargs):
        val_func = self.func(*args, **kwargs)
        val_encrypted = val_func*(self.c)/(self.d)
        print(f"Below is the encrypted result of {self.func.__name__}: {val_encrypted}.")

@funcEncryption
def myfunc_class2(*nums):
    print("I'm the second function to be decorated by a class decorator.\n")
    val_sum = sum(nums)
    return val_sum

myfunc_class2(1,2,3,4)

I'm the second function to be decorated by a class decorator.

Below is the encrypted result of myfunc_class2: 7.5.


### 以上，完成了用类装饰器为函数计算结果加密的操作。可见，类装饰器的初始化方法就相当于函数装饰器的外层，
### 用于传入待装饰的函数，以及定义一些后续操作需要使用的属性。
### __call__魔法方法则相当于函数装饰器内层的wrapper, 用于定义所需添加的功能。

### 下面还有四种排列组合的最后一种，即用类装饰器来装饰类。仍以为学生信息类添加额外属性为例，如下：

In [77]:
class classDecorator:
    def __init__(self,cls):
        self.cls = cls
        self.height_cm = 165
        self.weight_kg = 60
    def __call__(self, *args, **kwargs):
        object_new = self.cls(*args, **kwargs)
        object_new.height_cm = self.height_cm
        object_new.weight_kg = self.weight_kg
        print(f"Below are the attributes of the newly decorated object:\n")
        print(object_new.__dict__)
        return object_new

@classDecorator
class studentInfo_3:
    def __init__(self,name,gender,age):
        self.name = name
        self.gender = gender
        self.age = age
    def __call__(self):
        pass

student_5 = studentInfo_3("Jokky","male",24)
student_5()
type(student_5)

Below are the attributes of the newly decorated object:

{'name': 'Jokky', 'gender': 'male', 'age': 24, 'height_cm': 165, 'weight_kg': 60}


__main__.studentInfo_3

### 可见类装饰器装饰类的代码与装饰函数的代码有相似之处，只是把初始化方法中传入的待装饰参数换成了待装饰的类。
### 下面来实现一下Jokky兄弟的新要求：写出带参数的装饰器，根据当日的日期给出节日祝福（如果当天是节日的话）。
### 以函数装饰器加密函数运算结果为例，向装饰器中传递日期（字符串）为参数，在结果后打印出节日祝福。
### 正好今天是平安夜（先设定这么几个节日吧：新年，复活节，独立日，感恩节，平安夜，圣诞节。先这样。）
#### 注意：复活节和感恩节其实没有固定的日期，因为是根据星期来算的。我懒，这里就分别设为4月4日和11月27日。

In [7]:
import datetime
def encryption_greetings(date):
    dict_vacations = {"01-01": "Happy New Year!",
                      "04-04": "Happy Easter!",
                      "07-04": "Enjoy the Day of Independence!",
                      "11-27": "Happy Thanksgiving!",
                      "12-24": "Merry Christmas Eve!",
                      "12-25": "Merry Christmas!"}
    def encryption_dec(func):
        c = 150
        d = 200
        def wrapper(*nums):
            val_func = func(*nums)
            val_encrypted = val_func*c/d
            '''
            str_returned = "My ENCRYPTED returned value is %.2f."%(val_encrypted)
            return str_returned
            '''
            print(f"My ENCRYPTED returned value is {val_encrypted}.")
            if date in dict_vacations:
                print(dict_vacations[date])
        return wrapper
    return encryption_dec

date_today_formatted = datetime.date.today().strftime("%m-%d")
@encryption_greetings(date_today_formatted)
def myfunc_sum_return(*nums):
    print("I'm the second function to be decorated.\n"
          "My returned value is to be encrypted.\n")
    val_sum = sum(nums)
    return val_sum

myfunc_sum_return(1,2,3,4)

I'm the second function to be decorated.
My returned value is to be encrypted.

My ENCRYPTED returned value is 7.5.
Merry Christmas Eve!
